In [1]:
# This file is created by Zihao Wang, modified by Yingrui Ma.
## Load the model

In [2]:
import sys
import ast
sys.path.append("..")
sys.path.append("../src")

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import segmentation_models as sm
sm.set_framework('keras')
import ad_seg_utils as seg_utils
import bb_eval_utils as eval_utils
import csv
import math
import albumentations as A

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [3]:
SEGMENTATION = "skin"
PERTURBATION = "base"
print("Program initiating... type of segmentation: " + SEGMENTATION + ", type of perturbation: " + PERTURBATION)

BIN_SEG = False
CLASSES = ['background', 'skin']
WEIGHTS = np.array([1, 1])
target_idx = 1
MODEL_NAME = '/skin_base/base_skin_LR1.h5'
BACKBONE = 'efficientnetb3'
preprocess_input = sm.get_preprocessing(BACKBONE)

# set parameters based on the type of segmentation
if SEGMENTATION == 'SKIN' or SEGMENTATION == 'skin':
    pass
elif SEGMENTATION == 'AD' or SEGMENTATION == 'ad':
    BIN_SEG = False
    target_idx = 2
    CLASSES = ['background', 'skin', 'eczema']
    WEIGHTS = np.array([1, 1, 1])
    MODEL_NAME = '/AD_base/base_AD_LR1.h5'
else:
    print('Unexpected type of segmentation, should be either skin or ad\n program terminated')



"""# Model Evaluation"""
PROJ_DIR = "/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master"
OUTPUT_DIR = os.path.join(PROJ_DIR, 'output_trained_on_S+T')
PRED_DIR = os.path.join(OUTPUT_DIR, 'predictions/' + "test")
BB_DIR = os.path.join(OUTPUT_DIR, 'bounding_boxes')
EVAL_DIR = os.path.join(OUTPUT_DIR, 'evaluations')
MODEL_DIR = os.path.join(OUTPUT_DIR, 'models')
DATA_DIR = os.path.join(PROJ_DIR, 'data')

# new dataset
x_test_dir = os.path.join(DATA_DIR, 'dataset_combined/test_set_combined/reals')
y_test_dir = os.path.join(DATA_DIR, 'dataset_combined/test_set_combined/labels')

print('reading test images from: ' + str(x_test_dir))
print('reading test masks from: ' + str(y_test_dir))

test_dataset = seg_utils.Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=CLASSES, 
    augmentation=None,
    preprocessing=seg_utils.get_preprocessing(preprocess_input),
    is_train=False,
    use_full_resolution=False,
    binary_seg=BIN_SEG,
)
eczema_dataset = seg_utils.Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=['background', 'skin', 'eczema'],
    augmentation=None,
    preprocessing=seg_utils.get_preprocessing(preprocess_input),
    is_train=False,
    use_full_resolution=False,
    binary_seg=0,
)


model = seg_utils.load_model(dir=MODEL_DIR+MODEL_NAME, classes=CLASSES, weights=WEIGHTS)
print('Trained model loaded!')

Program initiating... type of segmentation: skin, type of perturbation: base
reading test images from: /rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/dataset_combined/test_set_combined/reals
reading test masks from: /rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/dataset_combined/test_set_combined/labels
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-08-05 20:36:24.437053: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-08-05 20:36:24.455925: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-05 20:36:24.455955: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: cx1-106-6-2.cx1.hpc.ic.ac.uk
2022-08-05 20:36:24.455966: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: cx1-106-6-2.cx1.hpc.ic.ac.uk
2022-08-05 20:36:24.456067: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.60.2
2022-08-05 20:36:24.456100: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.60.2
2022-08-05 20:36:24.456117: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.60.2
2022-08-05 20:36:24.457095

Trained model loaded!


## Example to predict mask, generate box and produce square crops

In [4]:
# Find refno and visno
dataset_name = 'TLA'

if dataset_name == 'TLA':
    ori_file_dir = DATA_DIR +'/mergedFile.csv'
    # Note that one image from id 39 has no score, it should be deleted manually
    no_score_id = ['33', '65', '66']

elif dataset_name == 'SWET':
    ori_file_dir = '/rds/general/user/ym521/projects/eczemanet/live/segmentation_images_ZW/skin_data/meta_data_skin_data.csv'


ori_file = pd.read_csv(ori_file_dir)


def find_ref_vis_no(image_file):
    if dataset_name == 'TLA':
        image_file = image_file[:-5]
        file_column = ori_file['image']
        for (index, colname) in enumerate(file_column):
            if image_file == colname[11:-4] or image_file == colname[11:-5]:
                if str(ori_file['id'].get(index)) in no_score_id:
                    return None, None
                else:  
                    return str(ori_file['id'].get(index)), str(ori_file['visno'].get(index))
        return None, None
            
    elif dataset_name == 'SWET':
        image_file = image_file[:-4]
        file_column = ori_file['filename']
        for (index, colname) in enumerate(file_column):
            if image_file == colname[0:6] + colname[12:-11]:  
                return str(ori_file['refno'].get(index)), str(ori_file['visno'].get(index))
        return None, None
    
    else:
        print('Please check variable <dataset_name>')
        return None, None
            
    



In [8]:
crop_type = 'manual'

if crop_type == 'auto':

    folder_dir = '/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/dataset_SWET/training_set_corrected_SWET/reals'

    # gt_dir = '/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA_masked'

    fns = os.listdir(folder_dir)
    i = 1
    total_num = len(fns)

    for file in fns:
    #     if '6963f21d-2a92-498a-9a80-6968e0ae95a5' in file:
    #         seg_utils.run_sigle_pred(model, os.path.join(folder_dir, file), "/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/output_trained_on_S+T/predictions/test", file, target_idx=target_idx, resize_ratio=0.4, preprocessing=seg_utils.get_preprocessing(preprocess_input), refno='39',visno='1')
    #         print('done')   
    #     print(os.path.join(folder_dir, file))

        print('Running ', i, ' of ', total_num, ', auto crops.')
        refno, visno = find_ref_vis_no(file)
        if refno and visno:
            seg_utils.run_sigle_pred(model, os.path.join(folder_dir, file), "/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/output_trained_on_S+T/predictions/test", file, target_idx=2, resize_ratio=0.4, preprocessing=seg_utils.get_preprocessing(preprocess_input), refno=refno,visno=visno)
    #         seg_utils.run_gt_image_combination_only_remove_background(file, os.path.join(folder_dir, file), gt_dir, "/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/output_trained_on_S+T/predictions/test", refno=refno, visno=visno)
        else:
            print('No score for file: ', file)
            print('--------------------------------------------------------')
            # For SWET dataset, the following images have no score:
            # '01040_wk16.jpg', '01047_wk04_3.jpg', '01040_wk00_2.jpg', '01041_wk12.jpg', '01044_wk12.jpg'
            # '05076_wk16_2.jpg', '05060_wk16.jpg', '03048_wk00.JPG', '05019_wk12_2.jpg', '05042_wk04.jpg'
            # '03038_wk16.JPG', '05011_wk04_2.jpg', '03123_wk00.JPG', '08056_wk00.jpg', '05063_wk00.jpg'
        i += 1
        
elif crop_type == 'manual':
    refno_list = []
    folder_dir = '/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/dataset_TLA/validation_set_TLA/reals'
    gt_dir = '/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA_masked'
    fns = os.listdir(folder_dir)
    i = 1
    total_num = len(fns)

    for file in fns:
        print('Running ', i, ' of ', total_num, ', manual crops.')
        refno, visno = find_ref_vis_no(file)
        refno_list.append(refno)
        if refno and visno:
            seg_utils.run_gt_image_combination_only_remove_background(file, os.path.join(folder_dir, file), gt_dir, "/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/output_trained_on_S+T/predictions/test", refno=refno, visno=visno)
        else:
            print('No score for file: ', file)
            print('--------------------------------------------------------')

        i += 1
    print(list(set(refno_list)))

        
else:
    print('Please check variable <crop_type>')
  

print('Finished!')

Running  1  of  109 , manual crops.
97a217c4-5c8e-425b-95a6-31763e429d7b.png : cropping done!
Running  2  of  109 , manual crops.
77b3e46f-124d-47cd-8c41-2dc6c17b8cf8.png : cropping done!
Running  3  of  109 , manual crops.
e3db7394-dcc0-47bd-8d10-452aead17321.JPG : cropping done!
Running  4  of  109 , manual crops.
f209c397-f7a6-4b8d-8bbe-9e2bf46ea19a.png : cropping done!
Running  5  of  109 , manual crops.
c7f81de8-21cf-4e25-a471-ce5ef36a807e.JPG : cropping done!
Running  6  of  109 , manual crops.
500890e0-1b91-4aaa-bbcb-693f339eea82.png : cropping done!
Running  7  of  109 , manual crops.
c9ed2b22-cb40-4af7-b464-8dd728eed66c.png : cropping done!
Running  8  of  109 , manual crops.
897dd53b-45fc-4191-bc2a-a85a464cd482.png : cropping done!
Running  9  of  109 , manual crops.
a17f3cde-8b5f-48a6-a7b0-cc01ca0111aa.png : cropping done!
Running  10  of  109 , manual crops.
54271878-ca59-49c6-8ff1-50a2335d0df3.png : cropping done!
Running  11  of  109 , manual crops.
9c769123-9f12-4928-a81

## Pre processing of severity training: csv files

In [6]:

# if dataset_name == 'TLA':
#     # Be careful with variable <crop_type>!
#     crop_dir = os.path.join(OUTPUT_DIR, 'predictions/' + crop_type + '_crops_TLA/crops')     
        
#     ori_file_dir = DATA_DIR +'/mergedFile.csv'
#     ori_file = pd.read_csv(ori_file_dir)
#     fns = os.listdir(crop_dir)
#     if len(fns) > 1:
#         i = 1
#         total_num = len(fns)
#         find = False

#         with open(DATA_DIR + '/meta_skin_' + crop_type + '_data_TLA.csv','w', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(['refno', 'visno', 'sassad_ery', 'sassad_exu', 'sassad_dry', 'sassad_cra', 'sassad_lic', 'sassad_tot', 'tiss_ery', 'tiss_exc', 'tiss_oed', 'tiss_tot', 'filename', 'filepath'])
#             for file in fns:
#                 crop_file = file[9:-11]
#                 # For some images, there could be more than 10 crops ('10' - '99' needs two char)
#                 crop_file_long = file[9:-12]
#                 file_column = ori_file['image']    
#                 for (index, colname) in enumerate(file_column):
#                     if crop_file == str(colname[11:-4]) or crop_file == str(colname[11:-5]) or crop_file_long == str(colname[11:-4]) or crop_file_long == str(colname[11:-5]):
#                         writer.writerow([ori_file['id'].get(index), ori_file['visno'].get(index), int(ori_file['ery'].get(index)[-1]), int(ori_file['exu'].get(index)[-1]), \
#                                         int(ori_file['dry'].get(index)[-1]), int(ori_file['cra'].get(index)[-1]), int(ori_file['lic'].get(index)[-1]), None, int(ori_file['ery'].get(index)[-1]), \
#                                         int(ori_file['exc'].get(index)[-1]), int(ori_file['oed'].get(index)[-1]), None, file, crop_dir + '/' + file])
#                         find = True
#                 if not find:
#         #             print(ori_file['image'].get(316)[11:-4])
#                     print('Did not find file: ', file)
#                 i += 1
#                 find = False
#     else:
#         print('Please check variable <crop_type>')

                

        
    
# elif dataset_name == 'SWET':
#     ori_file_dir = '/rds/general/user/ym521/projects/eczemanet/live/segmentation_images_ZW/skin_data/meta_data_skin_data.csv'
#     ori_file = pd.read_csv(ori_file_dir)
#     crop_dir = os.path.join(OUTPUT_DIR, 'predictions/auto_crops_SWET/crops')
#     fns = os.listdir(crop_dir)
#     i = 1
#     total_num = len(fns)
#     find = False
    
#     with open(DATA_DIR + '/meta_skin_auto_data_SWET.csv','w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['refno', 'visno', 'sassad_ery', 'sassad_exu', 'sassad_dry', 'sassad_cra', 'sassad_lic', 'sassad_tot', 'tiss_ery', 'tiss_exc', 'tiss_oed', 'tiss_tot', 'filename', 'filepath'])
#         for file in fns:
#             crop_file = file[11:-11]
#             file_column = ori_file['filename']    
#             for (index, colname) in enumerate(file_column):
#                 if crop_file == str(colname[0:6] + colname[12:-11]):
#                     writer.writerow([ori_file['refno'].get(index), ori_file['visno'].get(index), int(ori_file['sassad_ery'].get(index)), int(ori_file['sassad_exu'].get(index)), \
#                                     int(ori_file['sassad_dry'].get(index)), int(ori_file['sassad_cra'].get(index)), int(ori_file['sassad_lic'].get(index)), int(ori_file['sassad_tot'].get(index)), int(ori_file['tiss_ery'].get(index)), \
#                                     int(ori_file['tiss_exc'].get(index)), int(ori_file['tiss_oed'].get(index)), int(ori_file['tiss_tot'].get(index)), file, crop_dir + '/' + file])
#                     find = True
#                     break
#             if not find:
#                 print('Did not find file: ', file)
#             i += 1
#             find = False
# else:
#     print('Please check variable <dataset_name>')

    
    